In [1]:
import itertools
import sys

# import numpy
import pandas as pd

from cppy import cnf_to_pysat

from cppy import *
from cppy.model_tools.to_cnf import *


# pysat imports
from pysat.formula import CNF
from pysat.solvers import Solver

In [2]:

# Relation between 'rows' and 'cols', Boolean Variables in a pandas dataframe
class Relation(object):
    # rows, cols: list of names
    def __init__(self, rows, cols, name=""):
        self.cols = cols
        self.rows = rows
        self.name = name
        rel = BoolVar((len(rows), len(cols)))
        self.df = pd.DataFrame(index=rows, columns=cols)
        for i,r in enumerate(rows):
            for j,c in enumerate(cols):
                self.df.loc[r,c] = rel[i,j]
    # use as: rel['a','b']
    def __getitem__(self, key):
        try:
            return self.df.loc[key]
        except KeyError:
            print(f"Warning: {self.name}{key} not in this relation")
            return False


def exactly_one(lst):
    # return sum(lst) == 1
    # (l1|l2|..|ln) & (-l1|-l2) & ...
    allpairs = [(-a|-b) for a, b in itertools.combinations(lst, 2)]
    return [any(lst)] + allpairs


def exactly_one_at_most(lst):
    allpairs = [(-a|-b) for a, b in itertools.combinations(lst, 2)]
    return any(lst), allpairs


def buildBijectivity(rels):
    bij = []
    bv_bij = []
    for rel in rels:
        # bijection for all columns inside relation
        bv1 = BoolVar()
        bv2 = BoolVar()
        # for each relation
        for col_ids in rel.df:
            # one per column
            atleast, atmost = exactly_one_at_most(rel[:, col_ids])
            [bij.append(implies(bv1, clause)) for clause in atmost]
            bij.append(implies(bv2, atleast))
        bv_bij.append(bv1)
        bv_bij.append(bv2)

        # bijection for all rows inside relation
        bv3 = BoolVar()
        bv4 = BoolVar()
        for (_,row) in rel.df.iterrows():
            # one per row
            atleast, atmost = exactly_one_at_most(row)
            [bij.append(implies(bv3, clause)) for clause in atmost]
            bij.append(implies(bv4, atleast))
        bv_bij.append(bv3)
        bv_bij.append(bv4)
    return bij, bv_bij

In [3]:
def cost_puzzle(U, I, cost_clue):
    """
    U = user variables
    I = initial intepretation

    bij/trans/clues = subset of user variables w/ specific cost.
    """
    litsU = set(abs(l) for l in U) | set(-abs(l) for l in U)

    I0 = set(I)

    def cost_lit(lit):
        if lit not in litsU:
            raise CostFunctionError(U, lit)
        elif lit in cost_clue:
            return cost_clue[lit]
        else:
            # lit in
            return 1

    return cost_lit

In [4]:
if True:
    year = ["21", "30", "42", "47", "58"]
    orbital_period = ["orbital_period_1", "orbital_period_2", "orbital_period_3", "orbital_period_4", "orbital_period_5"]
    comet = ["the_other_comet", "gostroma", "trosny", "casputi", "sporrin"]
    type1 = ["the_other_type1", "whitaker", "tillman", "underwood", "parks"]
    cycle = ["2008", "2009", "2010", "2011", "2012"]

    types = [year, orbital_period, comet, type1, cycle]
    n = len(types)
    m = len(types[0])
    assert all(len(types[i]) == m for i in range(n)), "all types should have equal length"

    of = Relation(orbital_period, year, "of")
    has = Relation(comet, orbital_period, "has")
    discovered_by = Relation(comet, type1, "discovered_by")
    discovered_in = Relation(comet, cycle, "discoverd_in")
    is_linked_with_1 = Relation(year, comet, "is_linked_with_1")
    is_linked_with_2 = Relation(year, type1, "is_linked_with_2")
    is_linked_with_3 = Relation(year, cycle, "is_linked_with_3")
    is_linked_with_4 = Relation(orbital_period, type1, "is_linked_with_4")
    is_linked_with_5 = Relation(orbital_period, cycle, "is_linked_with_5")
    is_linked_with_6 = Relation(type1, cycle, "is_linked_with_6")

    rels = [of,
            has,
            discovered_by,
            discovered_in,
            is_linked_with_1,
            is_linked_with_2,
            is_linked_with_3,
            is_linked_with_4,
            is_linked_with_5,
            is_linked_with_6
    ]

    relStrs = [
        "of",
        "has",
        "discovered_by",
        "discovered_in",
        "is_linked_with_1",
        "is_linked_with_2",
        "is_linked_with_3",
        "is_linked_with_4",
        "is_linked_with_5",
        "is_linked_with_6"
    ]

    # Bijectivity
    bij, bv_bij = buildBijectivity(rels)

    # Transitivity
    trans = []
    bv_trans =  [BoolVar() for i in range(27)]

    for x in orbital_period:
        for y in year:
            for z in type1:
                t0 = to_cnf(implies(is_linked_with_4[x, z] & is_linked_with_2[y, z], of[x, y]))
                [trans.append(implies(bv_trans[0], clause)) for clause in t0]

                t1 = to_cnf(implies(~is_linked_with_4[x, z] & is_linked_with_2[y, z], ~of[x, y]))
                [trans.append(implies(bv_trans[1], clause)) for clause in t1]

                t2 = to_cnf(implies(is_linked_with_4[x, z] & ~is_linked_with_2[y, z], ~of[x, y]))
                [trans.append(implies(bv_trans[2], clause)) for clause in t2]

    for x in orbital_period:
        for y in year:
            for z in cycle:
                t3 = to_cnf(implies(is_linked_with_5[x, z] & is_linked_with_3[y, z], of[x, y]))
                [trans.append(implies(bv_trans[3], clause)) for clause in t3]

                t4 = to_cnf(implies(~is_linked_with_5[x, z] & is_linked_with_3[y, z], ~of[x, y]))
                [trans.append(implies(bv_trans[4], clause)) for clause in t4]

                t5 = to_cnf(implies(is_linked_with_5[x, z] & ~is_linked_with_3[y, z], ~of[x, y]))
                [trans.append(implies(bv_trans[5], clause)) for clause in t5]

    for x in comet:
        for y in orbital_period:
            for z in type1:
                t6 = to_cnf(implies(discovered_by[x, z] & is_linked_with_4[y, z], has[x, y]))
                [trans.append(implies(bv_trans[6], clause)) for clause in t6]

                t7 = to_cnf(implies(~discovered_by[x, z] & is_linked_with_4[y, z], ~has[x, y]))
                [trans.append(implies(bv_trans[7], clause)) for clause in t7]

                t8 = to_cnf(implies(discovered_by[x, z] & ~is_linked_with_4[y, z], ~has[x, y]))
                [trans.append(implies(bv_trans[8], clause)) for clause in t8]

    for x in comet:
        for y in orbital_period:
            for z in cycle:
                t9 = to_cnf(implies(discovered_in[x, z] & is_linked_with_5[y, z], has[x, y]))
                [trans.append(implies(bv_trans[9], clause)) for clause in t9]

                t10 = to_cnf(implies(~discovered_in[x, z] & is_linked_with_5[y, z], ~has[x, y]))
                [trans.append(implies(bv_trans[10], clause)) for clause in t10]

                t11 = to_cnf(implies(discovered_in[x, z] & ~is_linked_with_5[y, z], ~has[x, y]))
                [trans.append(implies(bv_trans[11], clause)) for clause in t11]
    
    for x in comet:
        for y in type1:
            for z in cycle:
                t12 = to_cnf(implies(discovered_in[x, z] & is_linked_with_6[y, z], discovered_by[x, y]))
                [trans.append(implies(bv_trans[12], clause)) for clause in t12]

                t13 = to_cnf(implies(~discovered_in[x, z] & is_linked_with_6[y, z], ~discovered_by[x, y]))
                [trans.append(implies(bv_trans[13], clause)) for clause in t13]

                t14 = to_cnf(implies(discovered_in[x, z] & ~is_linked_with_6[y, z], ~discovered_by[x, y]))
                [trans.append(implies(bv_trans[14], clause)) for clause in t14]

    for x in comet:
        for y in type1:
            for z in year:
                t15 = to_cnf(implies(is_linked_with_1[z, x] & is_linked_with_2[z, y], discovered_by[x, y]))
                [trans.append(implies(bv_trans[15], clause)) for clause in t15]

                t16 = to_cnf(implies(~is_linked_with_1[z, x] & is_linked_with_2[z, y], ~discovered_by[x, y]))
                [trans.append(implies(bv_trans[16], clause)) for clause in t16]

                t17 = to_cnf(implies(is_linked_with_1[z, x] & ~is_linked_with_2[z, y], ~discovered_by[x, y]))
                [trans.append(implies(bv_trans[17], clause)) for clause in t17]

    for x in comet:
        for y in cycle:
            for z in year:
                t18 = to_cnf(implies(is_linked_with_1[z, x] & is_linked_with_3[z, y], discovered_in[x, y]))
                [trans.append(implies(bv_trans[18], clause)) for clause in t18]

                t19 = to_cnf(implies(~is_linked_with_1[z, x] & is_linked_with_3[z, y], ~discovered_in[x, y]))
                [trans.append(implies(bv_trans[19], clause)) for clause in t19]

                t20 = to_cnf(implies(is_linked_with_1[z, x] & ~is_linked_with_3[z, y], ~discovered_in[x, y]))
                [trans.append(implies(bv_trans[20], clause)) for clause in t20]

    for x in year:
        for y in type1:
            for z in cycle:
                t21 = to_cnf(implies(is_linked_with_3[x, z] & is_linked_with_6[y, z], is_linked_with_2[x, y]))
                [trans.append(implies(bv_trans[21], clause)) for clause in t21]

                t22 = to_cnf(implies(~is_linked_with_3[x, z] & is_linked_with_6[y, z], ~is_linked_with_2[x, y]))
                [trans.append(implies(bv_trans[22], clause)) for clause in t22]

                t23 = to_cnf(implies(is_linked_with_3[x, z] & ~is_linked_with_6[y, z], ~is_linked_with_2[x, y]))
                [trans.append(implies(bv_trans[23], clause)) for clause in t23]

    for x in orbital_period:
        for y in type1:
            for z in cycle:
                t24 = to_cnf(implies(is_linked_with_5[x, z] & is_linked_with_6[y, z], is_linked_with_4[x, y]))
                [trans.append(implies(bv_trans[24], clause)) for clause in t24]

                t25 = to_cnf(implies(~is_linked_with_5[x, z] & is_linked_with_6[y, z], ~is_linked_with_4[x, y]))
                [trans.append(implies(bv_trans[25], clause)) for clause in t25]

                t26 = to_cnf(implies(is_linked_with_5[x, z] & ~is_linked_with_6[y, z], ~is_linked_with_4[x, y]))
                [trans.append(implies(bv_trans[26], clause)) for clause in t26]

    # clues
    clues = []
    bv_clues = [BoolVar() for i in range(11)]

    # 0. The comet discovered by Whitaker doesn't have an orbital period of 30 years
    # assumption_satisfied( 0  ) => ?a [comet]: discovered_by(a,whitaker) & ~ (?b [orbital_period]: of(b,30) & has(a,b)).
    c0a = []
    #for a in comet:
    #    subformula0 = any(
    #        of[b,"30"] & has[a,b]
    #        for b in orbital_period
    #    )
    #    c0a.append(discovered_by[a,"whitaker"] & ~ subformula0)
    # manual ALT
    for a in comet:
        subformula0 = any(
            of[b,"30"] & has[a,b]
            for b in orbital_period
        )
        #c0a.append(discovered_by[a,"whitaker"] & ~ subformula0)
        c0a.append(discovered_by[a,"whitaker"] & all(~of[b,"30"] | ~has[a,b] for b in orbital_period))

    for cl in to_cnf(any(c0a)):
        clues.append(implies(bv_clues[0], cl))
    print(0, len(clues))

    # 1. Gostroma was discovered 1 cycle after the comet discovered by Tillman
    # assumption_satisfied( 0  ) => ?c [cycle] d [comet] e [cycle]: discovered_by(d,tillman) & discovered_in(d,c) & e = c+1 & discovered_in(gostroma,e).
    c1a = []
    for c in cycle:
        for d in comet:
            for e in cycle:
                if int(e) ==  int(c) + 1:
                    c1a.append(discovered_by[d,"tillman"] & discovered_in[d,c] & discovered_in["gostroma",e])

    for cl in to_cnf(any(c1a)):
        clues.append(implies(bv_clues[1], cl))
    print(1, len(clues))
        
    # 2. Of the comet discovered by Underwood and the comet with an orbital period of 42 years, one was found in 2009 and the other is Trosny
    # assumption_satisfied( 0  ) => ?f [comet] g [comet] h [orbital_period]: discovered_by(f,underwood) & of(h,42) & has(g,h) & ~ (f = g) & (discovered_in(f,2009) & trosny = g | discovered_in(g,2009) & trosny = f).
    c2a = []
    for f in comet:
        for g in comet:
            for h in orbital_period:
                if not (f == g):
                    #discovered_by[f,"underwood"] & of[h,"42"] & has[g,h] & ((discovered_in[f,"2009"] & ("trosny" == g)) | (discovered_in[g,"2009"] & ("trosny" == f)))
                    if f == "trosny":
                        c2a.append( discovered_by[f,"underwood"] & of[h,"42"] & has[g,h] & discovered_in[g,"2009"] )
                    elif g == "trosny":
                        c2a.append( discovered_by[f,"underwood"] & of[h,"42"] & has[g,h] & discovered_in[f,"2009"] )
                

    for cl in to_cnf(any(c2a)):
        clues.append(implies(bv_clues[2], cl))
    print(2, len(clues))

    # 3. The comet with an orbital period of 21 years is either the comet discovered by Whitaker or Casputi
    # assumption_satisfied( 0  ) => ?i [comet] j [orbital_period]: of(j,21) & has(i,j) & ((?k [comet]: discovered_by(k,whitaker) & k = i) | casputi = i).
    c3a = []
    for i in comet:
        for j in orbital_period:
            #subformula3a = any(
            #    discovered_by[k,"whitaker"]
            #    for k in comet if k == i
            #)
            #of[j,"21"] & has[i,j] & ( discovered_by[i,"whitaker"] | ("casputi" == i))
            if i == "casputi":
                c3a.append( of[j,"21"] & has[i,j] )
            else:
                c3a.append( of[j,"21"] & has[i,j] & discovered_by[i,"whitaker"] )

    for cl in to_cnf(any(c3a)):
        clues.append(implies(bv_clues[3], cl))
    print(3, len(clues))

    # 4. The comet discovered in 2010 doesn't have an orbital period of 21 years
    # assumption_satisfied( 0  ) => ?l [comet]: discovered_in(l,2010) & ~ (?m [orbital_period]: of(m,21) & has(l,m)).
    c4a = []
    for l in comet:
        subformula4a = any(
            of[m,"21"] & has[l,m]
            for m in orbital_period
        )
        c4a.append(discovered_in[l,"2010"] & (~subformula4a))

    for cl in to_cnf(any(c4a)):
        clues.append(implies(bv_clues[4], cl))
    print(4, len(clues))

    # 5. The comet discovered by Tillman, the comet discovered in 2011 and Casputi are three different comets
    # assumption_satisfied( 0  ) => ?n [comet] o [comet]: ~ (n = o) & ~ (n = casputi) & ~ (o = casputi) & discovered_by(n,tillman) & discovered_in(o,2011).
    c5a = []
    for n in comet:
        for o in comet:
            if (not (n == o)) and (not (n == "casputi")) and (not (o == "casputi")):
                c5a.append(discovered_by[n,"tillman"] & discovered_in[o,"2011"])

    for cl in to_cnf(any(c5a)):
        clues.append(implies(bv_clues[5], cl))
    print(5, len(clues))

    # 6. Sporrin wasn't found in 2010
    # assumption_satisfied( 0  ) => ~ discovered_in(sporrin,2010).
    c6a = [~discovered_in["sporrin","2010"]]
    
    for cl in to_cnf(any(c6a)):
        clues.append(implies(bv_clues[6], cl))
    print(6, len(clues))

    # 7. Whitaker's comet was discovered in 2010
    # assumption_satisfied( 0  ) => ?p [comet]: discovered_in(p,2010) & discovered_by(p,whitaker).
    c7a = []
    for p in comet:
        c7a.append(discovered_in[p,"2010"] & discovered_by[p,"whitaker"])

    for cl in to_cnf(any(c7a)):
        clues.append(implies(bv_clues[7], cl))
    print(7, len(clues))
    
    # 8. The comet discovered by Parks was discovered 1 cycle before Whitaker's comet
    # assumption_satisfied( 0  ) => ?q [comet] r [cycle] s [comet] t [cycle]: discovered_by(q,parks) & discovered_in(s,r) & discovered_by(s,whitaker) & t = r-1 & discovered_in(q,t).
    c8a = []
    for q in comet:
        for r in cycle:
            for s in comet:
                for t in cycle:
                    if int(t) == int(r) - 1:
                        c8a.append(discovered_by[q,"parks"] & discovered_in[s,r] & discovered_by[s,"whitaker"]& discovered_in[q,t])

    for cl in to_cnf(any(c8a)):
        clues.append(implies(bv_clues[8], cl))
    print(8, len(clues))

    # 9. The comet discovered in 2011 doesn't have an orbital period of 47 years
    # assumption_satisfied( 0  ) => ?u [comet]: discovered_in(u,2011) & ~ (?v [orbital_period]: of(v,47) & has(u,v)).
    c9a = []
    for u in comet:
        subformula9 = any(
            of[v, "47"] & has[u,v]
            for v in orbital_period
        )
        c9a.append(discovered_in[u,"2011"] & ~subformula9)

    for cl in to_cnf(any(c9a)):
        clues.append(implies(bv_clues[9], cl))
    print(9, len(clues))

    # 10. The comet discovered by Underwood has an orbital period of either 47 or 58 years
    # assumption_satisfied( 0  ) => ?w [comet] x [orbital_period]: discovered_by(w,underwood) & (of(x,47) | of(x,58)) & has(w,x).
    c10a = []
    for w in comet:
        for x in orbital_period:
            c10a.append(discovered_by[w,"underwood"] & (of[x,"47"] | of[x,"58"]) & has[w,x])

    for cl in to_cnf(any(c10a)):
        clues.append(implies(bv_clues[10], cl))
    print(10, len(clues))

    clueTexts = [
        "The comet discovered by Whitaker doesn't have an orbital period of 30 years",
        "Gostroma was discovered 1 cycle after the comet discovered by Tillman",
        "Of the comet discovered by Underwood and the comet with an orbital period of 42 years, one was found in 2009 and the other is Trosny",
        "The comet with an orbital period of 21 years is either the comet discovered by Whitaker or Casputi",
        "The comet discovered in 2010 doesn't have an orbital period of 21 years",
        "The comet discovered by Tillman, the comet discovered in 2011 and Casputi are three different comets",
        "Sporrin wasn't found in 2010",
        "Whitaker's comet was discovered in 2010",
        "The comet discovered by Parks was discovered 1 cycle before Whitaker's comet",
        "The comet discovered in 2011 doesn't have an orbital period of 47 years",
        "The comet discovered by Underwood has an orbital period of either 47 or 58 years"
    ]


0 121
1 202
2 403
3 499
4 620
5 657
6 658
7 674
8 1175
9 1296
10 1472


In [5]:
if True:
    for rel in rels:
        rowNames = list(rel.df.index)
        columnNames = list(rel.df.columns)
        for r in rowNames:
            for c in columnNames:
                print(r, c, rel.df.at[r, c].name + 1)

orbital_period_1 21 1
orbital_period_1 30 2
orbital_period_1 42 3
orbital_period_1 47 4
orbital_period_1 58 5
orbital_period_2 21 6
orbital_period_2 30 7
orbital_period_2 42 8
orbital_period_2 47 9
orbital_period_2 58 10
orbital_period_3 21 11
orbital_period_3 30 12
orbital_period_3 42 13
orbital_period_3 47 14
orbital_period_3 58 15
orbital_period_4 21 16
orbital_period_4 30 17
orbital_period_4 42 18
orbital_period_4 47 19
orbital_period_4 58 20
orbital_period_5 21 21
orbital_period_5 30 22
orbital_period_5 42 23
orbital_period_5 47 24
orbital_period_5 58 25
the_other_comet orbital_period_1 26
the_other_comet orbital_period_2 27
the_other_comet orbital_period_3 28
the_other_comet orbital_period_4 29
the_other_comet orbital_period_5 30
gostroma orbital_period_1 31
gostroma orbital_period_2 32
gostroma orbital_period_3 33
gostroma orbital_period_4 34
gostroma orbital_period_5 35
trosny orbital_period_1 36
trosny orbital_period_2 37
trosny orbital_period_3 38
trosny orbital_period_4 39
t

In [6]:
for i,clue in enumerate([c0a,c1a,c2a,c3a,c4a,c5a,c6a,c7a,c8a,c9a,c10a]):
    print(i)
    x = cnf_to_pysat(to_cnf(clue))

0
1
2
3
4
5
6
7
8
9
10


In [7]:
for i,clue in enumerate(clues):
    print(i)
    x = cnf_to_pysat(to_cnf(clue))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
if True:
    clues_cnf = cnf_to_pysat(to_cnf(clues))
    bij_cnf = cnf_to_pysat(to_cnf(bij))
    trans_cnf = cnf_to_pysat(to_cnf(trans))

    hard_clauses = [c for c in clues_cnf + bij_cnf + trans_cnf]
    soft_clauses = []
    soft_clauses += [[bv1.name + 1] for bv1 in bv_clues]
    soft_clauses += [[bv1.name + 1]  for bv1 in bv_bij]
    soft_clauses += [[bv1.name + 1]  for bv1 in bv_trans]

    weights = {}
    weights.update({bv.name + 1: 100 for bv in bv_clues})
    weights.update({bv.name + 1: 60 for bv in bv_trans})
    weights.update({bv.name + 1: 60 for bv in bv_bij})

    explainable_facts = set()
    bvRels = {}
    for rel, relStr in zip(rels, relStrs):
        rowNames = list(rel.df.index)
        columnNames = list(rel.df.columns)

        # production of explanations json file
        for r in rowNames:
            for c in columnNames:
                bvRels[rel.df.at[r, c].name + 1] = {"pred" : relStr.lower(), "subject" : r.lower(), "object": c.lower()}

        # facts to explain
        for item in rel.df.values:
            explainable_facts |= set(i.name+1 for i in item)

    matching_table = {
        'bvRel': bvRels,
        'Transitivity constraint': [bv.name + 1 for bv in bv_trans],
        'Bijectivity': [bv.name + 1 for bv in bv_bij],
        'clues' : {
            bv.name + 1: clueTexts[i] for i, bv in enumerate(bv_clues)
        }
    }


In [9]:
def check_clue(clue, m):
    cnf = cnf_to_pysat(clue)
    return check_cnf(cnf, m)
def check_cnf(cnf, m):
    for cl in cnf:
        if not any((l in m) for l in cl):
            return False
    return True
#print(check_clue(clue1, m))

def flat(ll):
    return [e for l in ll for e in l]

In [10]:
o_clauses, o_assumptions, o_weights, o_user_vars, matching_table = hard_clauses, soft_clauses, weights, explainable_facts, matching_table

if True:
    o_cnf = CNF(from_clauses=o_clauses)
    U = o_user_vars | set(x for lst in o_assumptions for x in lst)
    I = set(x for lst in o_assumptions for x in lst)
    f = cost_puzzle(U, I, o_weights)
    with Solver(bootstrap_with=o_clauses + o_assumptions) as s:
        sat = s.solve()
        print(sat)
        prev = None
        for id, m in enumerate(s.enum_models()):
            print(f"{id}: model found")
            for i,clue in enumerate([c0a,c1a,c2a,c3a,c4a,c5a,c6a,c7a,c8a,c9a,c10a]):
                print("cl",i, len(clue), check_clue(clue, m))
            print("hard", check_cnf(o_clauses, m))
            print("soft", check_cnf(o_assumptions, m))
            if not prev is None:
                print("diff", sorted(set(prev) - set(m), key=lambda l: abs(l)))
                break
            prev = m


True
0: model found
cl 0 5 True
cl 1 20 True
cl 2 40 True
cl 3 25 True
cl 4 5 True
cl 5 12 True
cl 6 1 True
cl 7 5 True
cl 8 100 True
cl 9 5 True
cl 10 25 True
hard True
soft True
1: model found
cl 0 5 True
cl 1 20 True
cl 2 40 True
cl 3 25 True
cl 4 5 True
cl 5 12 True
cl 6 1 True
cl 7 5 True
cl 8 100 True
cl 9 5 True
cl 10 25 True
hard True
soft True
diff [-2, 5, 12, -15, 36, -38, -46, 48, -178, 179, 188, -189, -201, 204, 211, -214, -326, 329, -339, 340, 344, -356, 359, -371, 374, -386, 387, -388, 476, -479, 489, -490, -494, 506, -509, 521, -524, 536, -537, 538, -695, 704, -708, 709, 719, -725, 734, -740, 749, -755, 762, -763, 845, -854, 858, -859, -869, 875, -884, 890, -899, 905, -912, 913, -1075, 1078, 1105, -1108, -1150, 1153, 1180, -1183, -1225, 1227, -1229, 1255, -1257, 1259, -1300, 1303, 1330, -1333, -1374, 1376, 1378, 1404, -1406, -1408, -1444, 1453, 1474, -1483, -1519, 1528, 1549, -1558, -1594, 1602, -1604, 1624, -1632, 1634, -1669, 1678, 1699, -1708, -1743, 1745, 1753, 1773,

In [11]:
# -2: not orbital_period_1 30 2
# 5: orbital_period_1 58 5

In [12]:
print(c3a)

[and([BV0, BV25, BV51]), and([BV5, BV26, BV51]), and([BV10, BV27, BV51]), and([BV15, BV28, BV51]), and([BV20, BV29, BV51]), and([BV0, BV30, BV56]), and([BV5, BV31, BV56]), and([BV10, BV32, BV56]), and([BV15, BV33, BV56]), and([BV20, BV34, BV56]), and([BV0, BV35, BV61]), and([BV5, BV36, BV61]), and([BV10, BV37, BV61]), and([BV15, BV38, BV61]), and([BV20, BV39, BV61]), (BV0) and (BV40), (BV5) and (BV41), (BV10) and (BV42), (BV15) and (BV43), (BV20) and (BV44), and([BV0, BV45, BV71]), and([BV5, BV46, BV71]), and([BV10, BV47, BV71]), and([BV15, BV48, BV71]), and([BV20, BV49, BV71])]


In [13]:
x = [c3a[-5],c3a[-6]]
print(x)
to_cnf(any([c3a[-6],BoolVar()]))

[and([BV0, BV45, BV71]), (BV20) and (BV44)]


[(BV4121) or (BV4120),
 or([BV4121, BV20 == 0, BV44 == 0]),
 (BV4121 == 0) or (BV20),
 (BV4121 == 0) or (BV44)]

In [14]:
for i,clue in enumerate([c0a,c1a,c2a,c3a,c4a,c5a,c6a,c7a,c8a,c9a,c10a]):
    print(i, to_cnf(any(clue)))
    print("")

0 [or([BV4128, BV4135, BV4142, BV4149, BV4156]), or([BV4122 == 0, BV1 == 0, BV25 == 0]), (BV4122) or (BV1), (BV4122) or (BV25), or([BV4123 == 0, BV6 == 0, BV26 == 0]), (BV4123) or (BV6), (BV4123) or (BV26), or([BV4124 == 0, BV11 == 0, BV27 == 0]), (BV4124) or (BV11), (BV4124) or (BV27), or([BV4125 == 0, BV16 == 0, BV28 == 0]), (BV4125) or (BV16), (BV4125) or (BV28), or([BV4126 == 0, BV21 == 0, BV29 == 0]), (BV4126) or (BV21), (BV4126) or (BV29), or([BV4127, BV4122 == 0, BV4123 == 0, BV4124 == 0, BV4125 == 0, BV4126 == 0]), (BV4127 == 0) or (BV4122), (BV4127 == 0) or (BV4123), (BV4127 == 0) or (BV4124), (BV4127 == 0) or (BV4125), (BV4127 == 0) or (BV4126), or([BV4128, BV51 == 0, BV4127 == 0]), (BV4128 == 0) or (BV51), (BV4128 == 0) or (BV4127), or([BV4129 == 0, BV1 == 0, BV30 == 0]), (BV4129) or (BV1), (BV4129) or (BV30), or([BV4130 == 0, BV6 == 0, BV31 == 0]), (BV4130) or (BV6), (BV4130) or (BV31), or([BV4131 == 0, BV11 == 0, BV32 == 0]), (BV4131) or (BV11), (BV4131) or (BV32), or([BV4

In [15]:
print(to_cnf(clues))

[or([BV3692 == 0, BV3709, BV3716, BV3723, BV3730, BV3737]), or([BV3692 == 0, BV3703 == 0, BV1 == 0, BV25 == 0]), or([BV3692 == 0, BV3703, BV1]), or([BV3692 == 0, BV3703, BV25]), or([BV3692 == 0, BV3704 == 0, BV6 == 0, BV26 == 0]), or([BV3692 == 0, BV3704, BV6]), or([BV3692 == 0, BV3704, BV26]), or([BV3692 == 0, BV3705 == 0, BV11 == 0, BV27 == 0]), or([BV3692 == 0, BV3705, BV11]), or([BV3692 == 0, BV3705, BV27]), or([BV3692 == 0, BV3706 == 0, BV16 == 0, BV28 == 0]), or([BV3692 == 0, BV3706, BV16]), or([BV3692 == 0, BV3706, BV28]), or([BV3692 == 0, BV3707 == 0, BV21 == 0, BV29 == 0]), or([BV3692 == 0, BV3707, BV21]), or([BV3692 == 0, BV3707, BV29]), or([BV3692 == 0, BV3708, BV3703 == 0, BV3704 == 0, BV3705 == 0, BV3706 == 0, BV3707 == 0]), or([BV3692 == 0, BV3708 == 0, BV3703]), or([BV3692 == 0, BV3708 == 0, BV3704]), or([BV3692 == 0, BV3708 == 0, BV3705]), or([BV3692 == 0, BV3708 == 0, BV3706]), or([BV3692 == 0, BV3708 == 0, BV3707]), or([BV3692 == 0, BV3709, BV51 == 0, BV3708 == 0]), o

In [23]:
o_clauses, o_assumptions, o_weights, o_user_vars, matching_table = hard_clauses, soft_clauses, weights, explainable_facts, matching_table

ms = []

if True:
    o_cnf = CNF(from_clauses=o_clauses)
    U = o_user_vars | set(x for lst in o_assumptions for x in lst)
    I = set(x for lst in o_assumptions for x in lst)
    f = cost_puzzle(U, I, o_weights)
    with Solver(bootstrap_with=o_clauses + o_assumptions) as s:
        sat = s.solve()
        print(sat)
        prev = None
        for id, m in enumerate(s.enum_models()):
            print(f"{id}: model found")
            ms.append(m)
            if not prev is None:
                print("diff", sorted(set(prev) - set(m), key=lambda l: abs(l)))
                break
            prev = m


True
0: model found
1: model found
diff [-2, 5, 12, -15, 36, -38, -46, 48, -178, 179, 188, -189, -201, 204, 211, -214, -326, 329, -339, 340, 344, -356, 359, -371, 374, -386, 387, -388, 476, -479, 489, -490, -494, 506, -509, 521, -524, 536, -537, 538, -695, 704, -708, 709, 719, -725, 734, -740, 749, -755, 762, -763, 845, -854, 858, -859, -869, 875, -884, 890, -899, 905, -912, 913, -1075, 1078, 1105, -1108, -1150, 1153, 1180, -1183, -1225, 1227, -1229, 1255, -1257, 1259, -1300, 1303, 1330, -1333, -1374, 1376, 1378, 1404, -1406, -1408, -1444, 1453, 1474, -1483, -1519, 1528, 1549, -1558, -1594, 1602, -1604, 1624, -1632, 1634, -1669, 1678, 1699, -1708, -1743, 1745, 1753, 1773, -1775, -1783, -3320, 3329, -3335, 3344, -3348, 3349, 3359, -3365, 3372, -3373, -3380, 3389, 3470, -3479, 3485, -3494, 3498, -3499, -3509, 3515, -3522, 3523, 3530, -3539, 3732, -3734, 4011, -4015, 4021, -4025, 4031, 4032, -4035, -4036, 4041, -4045, 4051, -4055]


In [24]:
# print sol:
for v in ms[1]:
    if v in bvRels:
        print(v, bvRels[v]['subject'], bvRels[v]['pred'], bvRels[v]['object'])
# manual check: sats all..

2 orbital_period_1 of 30
9 orbital_period_2 of 47
15 orbital_period_3 of 58
18 orbital_period_4 of 42
21 orbital_period_5 of 21
27 the_other_comet has orbital_period_2
34 gostroma has orbital_period_4
38 trosny has orbital_period_3
45 casputi has orbital_period_5
46 sporrin has orbital_period_1
52 the_other_comet discovered_by whitaker
60 gostroma discovered_by parks
64 trosny discovered_by underwood
66 casputi discovered_by the_other_type1
73 sporrin discovered_by tillman
78 the_other_comet discovered_in 2010
82 gostroma discovered_in 2009
89 trosny discovered_in 2011
95 casputi discovered_in 2012
96 sporrin discovered_in 2008
104 21 is_linked_with_1 casputi
110 30 is_linked_with_1 sporrin
112 42 is_linked_with_1 gostroma
116 47 is_linked_with_1 the_other_comet
123 58 is_linked_with_1 trosny
126 21 is_linked_with_2 the_other_type1
133 30 is_linked_with_2 tillman
140 42 is_linked_with_2 parks
142 47 is_linked_with_2 whitaker
149 58 is_linked_with_2 underwood
155 21 is_linked_with_3 201

In [25]:
# print sol:
for v in ms[0]:
    if v in bvRels:
        print(v, bvRels[v]['subject'], bvRels[v]['pred'], bvRels[v]['object'])

5 orbital_period_1 of 58
9 orbital_period_2 of 47
12 orbital_period_3 of 30
18 orbital_period_4 of 42
21 orbital_period_5 of 21
27 the_other_comet has orbital_period_2
34 gostroma has orbital_period_4
36 trosny has orbital_period_1
45 casputi has orbital_period_5
48 sporrin has orbital_period_3
52 the_other_comet discovered_by whitaker
60 gostroma discovered_by parks
64 trosny discovered_by underwood
66 casputi discovered_by the_other_type1
73 sporrin discovered_by tillman
78 the_other_comet discovered_in 2010
82 gostroma discovered_in 2009
89 trosny discovered_in 2011
95 casputi discovered_in 2012
96 sporrin discovered_in 2008
104 21 is_linked_with_1 casputi
110 30 is_linked_with_1 sporrin
112 42 is_linked_with_1 gostroma
116 47 is_linked_with_1 the_other_comet
123 58 is_linked_with_1 trosny
126 21 is_linked_with_2 the_other_type1
133 30 is_linked_with_2 tillman
140 42 is_linked_with_2 parks
142 47 is_linked_with_2 whitaker
149 58 is_linked_with_2 underwood
155 21 is_linked_with_3 201